In [1]:
from datetime import datetime, timedelta
import requests 
import pandas as pd
import numpy as np
import sys

from scripts.Scrapper import NIFTY50

In [2]:
today=datetime.now()
if(today.isoweekday()!=1):
    date=datetime.strftime( today- timedelta(1), '%d%m%Y')
else:
    date=datetime.strftime( today- timedelta(3), '%d%m%Y')
date

'19092019'

## Download Volatility File

In [3]:
nifty50=NIFTY50()

In [4]:
print("Downloading volatility File for "+date)
r=requests.get("http://nseindia.com/archives/nsccl/volt/CMVOLT_"+date+".CSV", stream = True) 
with open("./data/yesterday.csv","wb") as file: 
    for chunk in r.iter_content(chunk_size=1024):
         if chunk: 
            file.write(chunk)

In [5]:
# print("Downloading Nifty50 list")
# r=requests.get("https://www.nseindia.com/content/indices/ind_nifty50list.csv", stream = True) 
# with open("data/nifty50.csv","wb") as file: 
#     for chunk in r.iter_content(chunk_size=1024):
#          if chunk: 
#             file.write(chunk) 

In [6]:
def strtofloat(x):
    if(x=="-"):
        return None
    s=x.split("-")
    #print(s)
    if(len(s)>1):
        return round(float(s[1])*(-100),2)
    else:
        return round(float(s[0])*(100),2)


In [7]:
yesterday=pd.read_csv("data/yesterday.csv")
yesterday=yesterday.drop(["Date",
                          "Underlying Close Price (A)",
                          "Underlying Previous Day Close Price (B)",
                          "Underlying Log Returns (C) = LN(A/B)",
                          "Previous Day Underlying Volatility (D)"]
                         ,axis=1)
yesterday.columns=["Symbol","Daily%","Yearly%"]
yesterday['Daily%']=yesterday['Daily%'].apply(lambda x: strtofloat(x))
yesterday['Yearly%']=yesterday['Yearly%'].apply(lambda x: strtofloat(x))
yesterday=yesterday.loc[yesterday.Symbol.apply(lambda x: nifty50.nifty50check(x))]
yesterday.sort_values(["Daily%","Yearly%"], axis = 0, ascending = False,inplace = True, na_position ='last')
yesterday=yesterday.reset_index(drop=True)

yesterday#yesterday[yesterday['Daily%']>=2]

,Symbol,Daily%,Yearly%
0,YESBANK,6.89,131.63
1,IBULHSGFIN,4.37,83.49
2,TATAMOTORS,4.08,77.95
3,ZEEL,3.16,60.37
4,BPCL,3.01,57.51
5,TATASTEEL,2.89,55.21
6,VEDL,2.79,53.30
7,JSWSTEEL,2.39,45.66
8,EICHERMOT,2.38,45.47
9,MARUTI,2.38,45.47


In [8]:
final_data=pd.merge(yesterday,nifty50.get_nifty50table())

In [9]:
balance=int(input("Enter current balance in account:"))

Enter current balance in account:500


In [10]:
final_data["MIS Max"]=final_data.apply(lambda x: balance*x["Margin"]//x["LTP"],axis=1)
final_data["CNC Max"]=final_data.apply(lambda x: balance//x["LTP"],axis=1)
print(final_data[final_data["Daily%"]>2])

        Symbol  Daily%  Yearly%       LTP  % Change  Margin  MIS Max  CNC Max
0      YESBANK    6.89   131.63     54.00    -15.76     5.0     46.0      9.0
1   IBULHSGFIN    4.37    83.49    400.00     -4.59     3.0      3.0      1.0
2   TATAMOTORS    4.08    77.95    124.20      2.01     9.0     36.0      4.0
3         ZEEL    3.16    60.37    309.15     -7.83     5.0      8.0      1.0
4         BPCL    3.01    57.51    379.00     -0.81    12.5     16.0      1.0
5    TATASTEEL    2.89    55.21    345.55     -3.55    12.5     18.0      1.0
6         VEDL    2.79    53.30    146.50     -2.14     5.0     17.0      3.0
7     JSWSTEEL    2.39    45.66    220.50     -0.41    12.5     28.0      2.0
8    EICHERMOT    2.38    45.47  15750.00     -1.75    12.5      0.0      0.0
9       MARUTI    2.38    45.47   5987.70     -1.76    12.5      1.0      0.0
10  INDUSINDBK    2.35    44.90   1280.50     -3.75    12.5      4.0      0.0
11  HEROMOTOCO    2.30    43.94   2534.90     -1.60    12.5     